# Probabilistic classifier of texts into spam / ham

## Intro

Here is a classical "complete the notebook" assignment. 

You can run all the cells in the notebook, and some of them you have to complete. 

The code you have to complete is marked with `#TODO` comments. The cells containing such code also contain assertions that you should fulfill. 

If the cells produce no errors, you can be pretty sure you do everything OK. 

Let's try it!

In [1]:
def square_root(x):
    """ This is a function that takes a non-negative numeric argument x and produces its square root. """
    # TODO: calculate the square root of x and put it into the y variable instead of None. 
    # If you are not sure, have a look on the list of Python basic operators
    # https://www.tutorialspoint.com/python/python_basic_operators.htm
    y = x**0.5
    return y

assert square_root(144) == 12

Now that you understand the format, let's have look at a [Naive Bayes classifier](https://en.wikipedia.org/wiki/Naive_Bayes_classifier) of short messages into spam and not-spam.

The main idea behind it is that $$P(spam|text) = \frac{P(spam)P(text|spam)}{P(text)}$$

You will have to implement this formula along with some hacks to make its application more robust.

![](https://pics.me.me/suppose-you-have-one-rabbit-now-suppose-someone-gives-you-21826742.png)

## Loading the data

The cell below loads the file with messages. 

If you run this notebook locally on Windows, you have to download the file manually. 

In [4]:
url = "https://raw.githubusercontent.com/avidale/ps4ds2019/master/homework/week1/spam_classifier/SMSSpamCollection"

The following cell imports some Python libraries. It is possible that you have some of them not installed (namely, `pandas`). In this case, you have to install them using package manager from command line. The command would look like `pip install pandas` or `conda install pandas`.

If you run this notebook from Google Colab, then the libraries are already installed

In [3]:
# load some useful Python libratries

import pandas as pd # the library for working with data tables
import re
from collections import Counter # a class for counting objects (words and text labels, in our case)

In [5]:
# load the data from disk to a tabular format, and give readable names to its columns
data = pd.read_csv(url, sep='\t', header=None)
data.columns = ['target', 'text']

In this dataset, "ham" is a good text, and "spam" is, well, spam. 

In [7]:
# enable pandas to display large texts and look into our data
pd.options.display.max_colwidth = 300

print(data.shape) # number of rows and columns
data.head(5)

(5572, 2)


,target,text
0,ham,"Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives around here though"


## Preprocessing the data

In a minute we will have to estimate probabilites of different texts. 

We could use *language models* using e.g. n-grams or recurrent neural networks, to calculate probability of original texts. 

But for our problem, it will be sufficient to represent each text with the set of words (and other symbols) that occur in it. This representation ignores word order and number of words.

That is, we will not make difference between texts 

> this one is a long message. 

and 

> this message is a long long long long long long one.

Both will be represented as a set of tokens:

In [31]:
def get_words(text):
    """ This function converts the given text into an unordered and uncounted bag of words. """
    return set(re.split('\W+', text)).difference({''})

# just an example
get_words("this message is a long, long, long, long long long one.")

{'a', 'is', 'long', 'message', 'one', 'this'}

This simplified approach will allow us to train the probabilistic model of texts using a modest amount of data.

In [9]:
# apply this logic to texts of all messages
bags_of_words = [get_words(text) for text in data.text]

To evaluate how well our model classifies messages, let's train it on the first 3000 texts, and measure accuracy on the rest.

In [10]:
n_train = 3000
train_x, test_x, train_y, test_y = bags_of_words[:n_train], bags_of_words[n_train:], data.target[:n_train], data.target[n_train:]

## The basic classifier

In the cell below, we will count occurences of words under different labels.

We are going to use `Counter` objects. If you are not sure how they work, please look at [the documentation](https://docs.python.org/3.6/library/collections.html#collections.Counter). 

In [11]:
# label

In [12]:
# words

In [13]:
# this counter will keep the number of spam and ham texts
label_counter = Counter()

# these counters will keep the frequency of each word in ham and spam texts
word_counters = {
    'spam': Counter(), 
    'ham': Counter()
}

all_words = set()

for label, words in zip(train_y, train_x):
    all_words.update(words)
    # TODO: use the `update` methods of all 3 counters, to calculate total number of  DONE
    label_counter.update([label])
    word_counters[label].update(words)
assert label_counter['spam'] == 409
assert word_counters['ham']['hello'] >= 2

In [14]:
type(all_words)

set

In [16]:
word_counters['spam']['Have']

2

In [82]:
# word_counters['spam']

Now let's calculate different probabilities of words, texts, and labels for our classifier

In [17]:
def prior_probability_of_label(label):
    """ This function evaluates probability of the given label (it can be 'spam' or 'ham'), using the counters. """
    # TODO: calculate and relturn this probability as ratio of number of texts with this labels to number all texts
    return label_counter[label]/(label_counter["spam"]+label_counter["ham"])

assert round(prior_probability_of_label('spam'), 2) == 0.14
assert round(prior_probability_of_label('ham'), 2) == 0.86

In [18]:

def word_probability_given_label(word, label):
    """ This function calculates probability of a word occurence in text, conditional on the label of this text. """
    # TODO: calculate and return this probability 
    # as ratio of number of texts with this word and label to number of texts with this label
    return word_counters[label][word]/label_counter[label]

assert round(word_probability_given_label("99", "spam"), 3) == 0.002

In [20]:
word_probability_given_label("hello", "ham")

0.0007719027402547279

Here we encounter the first practical problem: some words have never occurred in our training data. 

But they can probably occur in the texts to which our model will be applied in the future. 

To assign a non-zero probability to such texts, we can slightly modify the `word_probability_given_label`. For example, instead of original estimate, 

$$\hat{p}(word|label) = \frac{count(word, label)}{count(label)}$$

we could use a "smoothed" version

$$\hat{p}(word|label) = \frac{count(word, label) + \alpha\times p}{count(label) + p}$$

where $alpha\in(0, 1)$ is the anchor probability towards which we move our estimate, and $p$ is the step size towards this anchor. 

Values like $p=0.1$ and $\alpha=1^{-3}$ would do.  

In [21]:
# TODO: modify the `word_probability_given_label` function, by moving each probability towards a small positive constant
def word_probability_given_label(word, label):
    """ This function calculates probability of a word occurence in text, conditional on the label of this text. """
    return ((word_counters[label][word])+(p*α))/(label_counter[label]+p)

p=0.1
α=1e-3
assert word_probability_given_label("999", "spam") > 0
assert word_probability_given_label("999", "spam") < 0.005

Now we can move from words to texts. 

Here is where we apply our naive assumption that occurrences of each word are independent:
$$ P(text|label) = \prod_{word \in text} P(word|label) \times \prod_{word \notin text} (1-P(word|label)) $$

In [22]:
def text_probability_given_label(text, label):
    """ This function calculates probability of the text conditional on its label. """
    if isinstance(text, str):
        text = get_words(text)
    probability = 1.0
    # TODO: calculate the probability of text given label. 
    # use a function defined above and the naive assumption of word independence
    for word in all_words:
        if word in text:
            probability = word_probability_given_label(word, label) * probability
        else:
            probability = (1-word_probability_given_label(word, label)) * probability
    return probability

greeting1 = 'hello how are you'
greeting2 = 'hello teacher how are you'

assert text_probability_given_label(greeting1, 'ham') > 0
assert text_probability_given_label(greeting1, 'ham') < 0.0001
assert text_probability_given_label(greeting2, 'ham') < text_probability_given_label(greeting1, 'ham')

In [ ]:
# def label_probability_given_text(label1, label2, text):
#     proba_label1 = prior_probability_of_label(label1)
#     proba_label2 = prior_probability_of_label(label2)
#     text_given_label1 =  text_probability_given_label(text, label1)
#     text_given_label2 =  text_probability_given_label(text, label2)
#     prob_text = proba_label1*text_given_label1 + proba_label2*text_given_label2
#     label_probability_given_text = proba_label1 * text_given_label1/prob_text
#     return(label_probability_given_text)

Now you have all the components to compile your first probabilistic classifier!

Now we need to find P of label given a text.

For that we first need to find P label given word.

For that we apply p(l/t) = ( p(l) * p(t/l) ) / p(t)



In [24]:
def label_probability_given_text(text, label):
    if label == 'ham':
        label1 = 'ham'
        label2 = 'spam'
    if label == 'spam':
        label1 = 'spam'
        label2 = 'ham'
    else:
        pass
    proba_label1 = prior_probability_of_label(label1)
    proba_label2 = prior_probability_of_label(label2)
    text_given_label1 =  text_probability_given_label(text, label1)
    text_given_label2 =  text_probability_given_label(text, label2)
    prob_text = proba_label1*text_given_label1 + proba_label2*text_given_label2
    label_probability_given_text = proba_label1 * text_given_label1/prob_text
    return(label_probability_given_text)

text1 = 'hello how r you'
text2 = 'only today you can buy our book with 50% discount!'


assert label_probability_given_text(text1, 'ham') + label_probability_given_text(text1, 'spam') == 1.0
assert label_probability_given_text(text1, 'ham') > label_probability_given_text(text1, 'spam')
assert label_probability_given_text(text1, 'ham') > label_probability_given_text(text2, 'ham')

In [ ]:
# def label_probability_given_word(word, label):
#     """ This function calculates probability of a word occurence in text, conditional on the label of this text. """
#     # print (1 -((word_counters[label][word])+(p*α))/(label_counter[label]+p))
#     return (1 -((word_counters[label][word])+(p*α))/(label_counter[label]+p))
#     # return ((label_counter[label])+(p*α))/(word_counters[label][word]+p)
#     # return ((word_counters[label][word])+(p*α))/(label_counter[label]+p)

## Tuning the classifier

Now we have the classifier, but we don't know how well it works on the unseen data. 

Let's see what fraction of test messages are classified correctly:

In [32]:
threshold = 0.8
alpha=1e-2
test_spam_probabilities = [label_probability_given_text(text, 'spam') for text in test_x]
test_predictions = ['spam' if spamness > threshold else 'ham' for spamness in test_spam_probabilities]

accuracy = sum(1 if pred == fact else 0 for pred, fact in zip(test_predictions, test_y)) / len(test_y)
print(accuracy)

assert accuracy > 0.9

0.9844479004665629


This is a good accuracy, but you can achieve better results by tuning the algorithm. 

What you can do:
* play with the different values of the threshold
* play with the regularization constants that you used in `word_probability_given_label`
* experiment with different implementations of `get_words` - e.g. ignore the word case, or use word lemmas
* use your imagination

Can you beat 99% accuracy?

Have a good time! (-: